<a href="https://colab.research.google.com/github/Juniorffonseca/python-valorant-preditor/blob/main/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fazendo meu projeto do TCC, que foi feito em R, na linguagem Python.
O projeto consiste em usar redes neurais para prever resultado de partidas do cenário profissional de Valorant


In [4]:
# Carregando pacotes
import requests
from bs4 import BeautifulSoup

In [5]:
# Criando variável páginas e criando variável 'p' que será a parte final do url (o número da página)
paginas = []
p = 1

# Criando um laço for que armazenará o url de cada página dentro da variável paginas
for i in range(1, 51):
    paginas.append(f'https://www.vlr.gg/matches/results/?page={p}')
    p += 1

# Variável partidas e variável c
partidas = []
c = 1

# Função que retorna o url de cada partida
def funcaoPagina(pagina):
    response = requests.get(pagina)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    partidas = [link.get('href') for link in links]
    partidas = partidas[14:64]
    partidas = ['https://www.vlr.gg' + partida for partida in partidas]
    return partidas


In [6]:
# Criando f e uma lista que receberá todos os returns da funcaoPagina (url de cada partida)
f = 0
a = []

# Executando um for que fará a iteração da funcaoPagina todas as vezes necessárias
for i in paginas:
    a.append(funcaoPagina(paginas[f]))
    f += 1

a

[['https://www.vlr.gg/167840/sharper-esport-vs-gelida-esports-challengers-league-thailand-split-1-w4',
  'https://www.vlr.gg/167800/boom-esports-vs-dewa-united-esports-challengers-league-indonesia-split-1-r4',
  'https://www.vlr.gg/164050/x10-esports-vs-todak-challengers-league-malaysia-singapore-split-1-d7',
  'https://www.vlr.gg/167658/crazy-raccoon-vs-sengoku-gaming-challengers-league-japan-split-1-w3',
  'https://www.vlr.gg/167839/xerxia-esports-vs-full-sense-challengers-league-thailand-split-1-w4',
  'https://www.vlr.gg/170380/team-big-baam-vs-fancy-united-esports-challengers-league-vietnam-split-1-w1',
  'https://www.vlr.gg/168299/rio-company-vs-dplus-esports-world-cyber-games-challengers-league-korea-split-1-w3',
  'https://www.vlr.gg/167799/dominatus-vs-arf-team-challengers-league-indonesia-split-1-r4',
  'https://www.vlr.gg/168478/puggers-vs-sky-empire-challengers-league-philippines-split-1-w3',
  'https://www.vlr.gg/170379/lazybutwannawin-vs-team-lofi-challengers-league-vietn